<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#replication-of-NPZD-model" data-toc-modified-id="replication-of-NPZD-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>replication of NPZD model</a></span><ul class="toc-item"><li><span><a href="#equations" data-toc-modified-id="equations-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>equations</a></span><ul class="toc-item"><li><span><a href="#Nutrients-in-bottom-layer-(N)" data-toc-modified-id="Nutrients-in-bottom-layer-(N)-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Nutrients in bottom layer (N)</a></span></li><li><span><a href="#phytoplankton-(P)" data-toc-modified-id="phytoplankton-(P)-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>phytoplankton (P)</a></span></li><li><span><a href="#zooplankton-(Z)" data-toc-modified-id="zooplankton-(Z)-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>zooplankton (Z)</a></span></li><li><span><a href="#detritivore-(D)" data-toc-modified-id="detritivore-(D)-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>detritivore (D)</a></span></li></ul></li><li><span><a href="#parameters" data-toc-modified-id="parameters-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>parameters</a></span></li></ul></li></ul></div>

# replication of NPZD model
## equations
### Nutrients in bottom layer (N)  
\begin{equation}
    N_0(z) = a_Nz + b_N
\end{equation}

growth rate
\begin{equation}
    N'(t) = \mu_PP + \beta(1-k_{NZ})(G_P + G_D) + m_DD + \dfrac{(\omega_{mix} + H'(t))(N_0-N)}{H}
\end{equation}

### phytoplankton (P)
\begin{equation}
    P'(t) = \mu_PP - G_P - m_PP - m_{P2}P^2 - \dfrac{\omega_{mix}+H'(t)P}{H(t)}
\end{equation}

phytoplankton growth rate
\begin{align}
    \mu_P &= \mu_P^{max}(T)L_N(N)L_I(I(t,z))\nonumber\\
    &= \dfrac{\mu_P^{max}(T)L_N(N)}{24H}\int_0^{24h}\int_0^HL_I(I(t,z))dzdt
\end{align}

photosynthetic rate with Q$_10$=1.895
\begin{equation}
    V_P^{max}(T) = V_P^{max}(0)1.066^T
\end{equation}

Monod nutrient limit relationship
\begin{equation}
    L_N(N)=\dfrac{N}{k_N+N}
\end{equation}

Beer's law for light availability
\begin{equation}
    I = I_0exp(-k_{PAR}z)
\end{equation}

traditional approach
\begin{equation}
    k_{PAR} = k_cP
\end{equation}

EMPOWER approach
\begin{equation}
    k_{PAR,i} = b_{0,i} + b_{1,i}C^{0.5} + b_{2,i}C + b_{3,i}C^{1.5} + b_{4,i}C^{2} + b_{5,i}C^{2.5}
\end{equation}

Simulated daylight P-I curve from Smith function
\begin{equation}
    V_P = \dfrac{\alpha IV_P^{max}}{\sqrt{(V_P^{max})^2+\alpha^2I^2}}
\end{equation}

Simulated daylight P-I curve from exponential function
\begin{equation}
    V_P = V_P^{max}(1-exp(-\alpha I/V_P^{max}))
\end{equation}

growth rate coefficient
\begin{equation}
    G_P = Z(\dfrac{I_{max}\hat{\varphi_P}P}{k_Z^2+\hat{\varphi_P}P+\hat{\varphi_D}D})
\end{equation}
\begin{equation}
    \hat{\varphi_P} = \varphi_PP
\end{equation}
\begin{equation}
    \varphi_P = \dfrac{k_Z^2}{k_P^2}
\end{equation}

### zooplankton (Z)

rate of change of zooplankton density
\begin{equation}
    Z'(t) = (\beta k_N(G_P + G_D)) - (m_ZZ + m_{Z2}Z^2) - \dfrac{Z(\omega_{mix} + H'(t))}{H(t)}
\end{equation}

### detritivore (D)

\begin{equation}
    G_D = Z(\dfrac{I_{max}\hat{\varphi_P}P}{k_Z^2+\hat{\varphi_P}P+\hat{\varphi_D}D})
\end{equation}
\begin{equation}
    \hat{\varphi_D} = \varphi_DD
\end{equation}
\begin{equation}
    \varphi_D = \dfrac{k_D^2}{k_P^2}
\end{equation}

growth rate
\begin{equation}
    D'(t) = m_PP + M_{P2}P^2 + m_ZZ + (1-\beta)(G_P + G_D) - G_D - m_DD - \dfrac{(\omega_{mix} + H'(t) + v_D)D}{H}
\end{equation}

## parameters
notation | unit | meaning
--- | --- | ---
$z$ | m | depth from water column surface
$t$ | day | daily time step
$N_0$ | mmol/m$^3$ | fixed concentration of nutrient
$a_N$ | mmol/m$^4$ | arbitrary coefficient determined from data
$b_N$ | mmol/m$^3$ | arbitrary coefficient determined from data
$H(t)$ | m | mixed layer depth at time $t$
$\mu_P$ | day$^{-1}$ | vertically mean temp-dependent daily growth rate
$\mu_P^{max}(T)$ | day$^{-1}$ | max $\mu_P$
$L_N(N)$ | [no] | non-dimensional nutrient limitation
$L_I(I(t,z))$ | [no] | non-dimensional light limitation
$V_P^{max}(T)$ | rate | max photosynthetic rate
$V_P^{max}(0)$ | rate | photosynthetic rate at 0$^o$C
$k_N$ | [no] | half-saturation constant
$I$ | lux | light intensity at depth $z$
$I_0$ | lux | light intensity at depth =0
$k_{PAR}$ | [no] | attenuation coefficient
$k_\omega$ | 0.04mmolN^{-1} | convention arbitrary number
$k_c$ | 0.03mmolN^{-1} | convention arbitrary number
$C$ | mmol/m$^3$ | chlorophyll concentration
$b_{1:5,i}$ | [no] | polynomial coefficients in the paper
$I_P$ | rate | ingestion rate of zooplankton (predator)
$\varphi_P$ | [no] | prey preference for zooplankton
$I_D$ | rate | ingestion rate of detritivore
$\varphi_D$ | [no] | prey preference for detritivore
$I_{max}$ | rate | max specific grazing rate
$k_P$ | [no] | half-saturation value for Type 3 single-prey response for phytoplankton ingestion
$\beta$ | [no] | absorption efficiency
$k_{NZ}$ | [no] | net production efficiency
$1-\beta$ | [no] | egestion
$\beta\cdot k_{NZ}$ | [no] | growth
$\beta(1-k_{NZ})$ | [no] | excretion
$m_D$ | rate | detritus remineralize rate
$v_D$ | rate | gravitation mixing